In [19]:
%load_ext autoreload
%autoreload 2
import torch
from cosypose.config import LOCAL_DATA_DIR
from cosypose.datasets.datasets_cfg import make_scene_dataset
from cosypose.rendering.bullet_scene_renderer import BulletSceneRenderer
from cosypose.visualization.singleview import make_singleview_prediction_plots, filter_predictions
from cosypose.visualization.singleview import filter_predictions
from bokeh.plotting import gridplot
from bokeh.io import show, output_notebook; output_notebook()
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [20]:
# result_id = 'tless-siso-n_views=1--684390594'
# ds_name, urdf_ds_name = 'tless.primesense.test', 'tless.cad'
# pred_key = 'pix2pose_detections/refiner/iteration=4'

result_id = 'ycbv-n_views=1--4282115980' # change to the result id under result folder
ds_name, urdf_ds_name = 'ycbv.test.keyframes', 'ycbv'
pred_key = 'posecnn_init/refiner/iteration=2'


results = LOCAL_DATA_DIR / 'results' / result_id / 'results.pth.tar'
scene_ds = make_scene_dataset(ds_name)
results = torch.load(results)['predictions']

18:30:08.301262 - Building index and loading annotations...


[Memory]66608.5s, 1110.1min: Loading build_index...


In [21]:
results[pred_key].infos.loc[:, ['scene_id', 'view_id']].groupby('scene_id').first()

,view_id
scene_id,
48,1104


In [27]:
# scene_id, view_id = 12, 103

# Replace this here, you can use the dataframe above to get examples of scene/view ids.
scene_id, view_id = 48, 1605

this_preds = filter_predictions(results[pred_key], scene_id, view_id)
renderer = BulletSceneRenderer(urdf_ds_name)
figures = make_singleview_prediction_plots(scene_ds, renderer, this_preds)
renderer.disconnect()
# print(this_preds)

show(figures['input_im'])
show(figures['pred_overlay'])

[48]


In [23]:
print(this_preds)

PandasTensorCollection(
    poses: torch.Size([5, 4, 4]) torch.float32 cpu,
    poses_input: torch.Size([5, 4, 4]) torch.float32 cpu,
    K_crop: torch.Size([5, 3, 3]) torch.float32 cpu,
    boxes_rend: torch.Size([5, 4]) torch.float32 cpu,
    boxes_crop: torch.Size([5, 4]) torch.float32 cpu,
    initial_bboxes: torch.Size([5, 4]) torch.float32 cpu,
----------------------------------------
    infos:
   scene_id  view_id  score       label  det_id  batch_im_id  group_id  \
0        48        1    1.0  obj_000001       0            0        47   
1        48        1   20.0  obj_000020       1            0        47   
2        48        1   14.0  obj_000014       2            0        47   
3        48        1   19.0  obj_000019       3            0        47   
4        48        1    6.0  obj_000006       4            0        47   

   batch_pred_id  
0            435  
1            436  
2            437  
3            438  
4            439  
)


In [24]:
print(this_preds.poses[1])

tensor([[-0.9804, -0.1969, -0.0013, -0.0251],
        [-0.0567,  0.2886, -0.9558,  0.0959],
        [ 0.1886, -0.9370, -0.2942,  0.6658],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
